In [ ]:
# This is a Random Forest Regression Model, 
# however when training the model the process will not complete
# Needs bug fixing

In [1]:
#Load cleaned_airline_data.csv into Spark Dataframe
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ml-delay').getOrCreate()
df = spark.read.csv('cleaned_airline_data_v5.csv', header = True, inferSchema = True)

2022-12-16T13:08:29,164 WARN [Thread-4] org.apache.spark.sql.SparkSession - Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
#show first five rows in dataframe. (note this has been transposed in order to easily view)
import pandas as pd
pd.DataFrame(df.take(5), columns=df.columns).transpose()

,0,1,2,3,4
FL_DATE,2013-07-14 00:00:00,2013-07-14 00:00:00,2013-07-14 00:00:00,2013-07-14 00:00:00,2013-07-14 00:00:00
OP_CARRIER,DL,DL,DL,DL,DL
OP_CARRIER_FL_NUM,1004,1005,1006,1007,1007
ORIGIN,ATL,MSP,JFK,ATL,MCI
DEST,ORF,AUS,BOS,MCI,ATL
CRS_DEPT_TIME,2005,1339,1200,1505,1700
DEPT_TIME,2013.0,1336.0,1155.0,1524.0,1703.0
DEP_DELAY,8,-3,-5,19,3
TAXI_OUT,18,10,16,17,10
WHEELS_OFF,2031.0,1346.0,1211.0,1541.0,1713.0


In [3]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
categoricalColumns = ["OP_CARRIER","OP_CARRIER_FL_NUM","ORIGIN","DEST"]

OP_CARRIER_indexer = StringIndexer(inputCol="OP_CARRIER", outputCol="OP_CARRIER_Index")
df_1 = OP_CARRIER_indexer.fit(df).transform(df)

onehotencoder_OP_CARRIER_vector = OneHotEncoder(inputCol="OP_CARRIER_Index", outputCol="OP_CARRIER_vec")
df_2 = onehotencoder_OP_CARRIER_vector.fit(df_1).transform(df_1)

OP_CARRIER_FL_NUM_indexer = StringIndexer(inputCol="OP_CARRIER_FL_NUM", outputCol="OP_CARRIER_FL_NUM_Index")
df_3 = OP_CARRIER_FL_NUM_indexer.fit(df_2).transform(df_2)

onehotencoder_OP_CARRIER_FL_NUM_vector = OneHotEncoder(inputCol="OP_CARRIER_FL_NUM_Index", outputCol="OP_CARRIER_FL_NUM_vec")
df_4 = onehotencoder_OP_CARRIER_FL_NUM_vector.fit(df_3).transform(df_3)

ORIGIN_indexer = StringIndexer(inputCol="ORIGIN", outputCol="ORIGIN_Index")
df_5 = ORIGIN_indexer.fit(df_4).transform(df_4)

onehotencoder_ORIGIN_vector = OneHotEncoder(inputCol="ORIGIN_Index", outputCol="ORIGIN_vec")
df_6 = onehotencoder_ORIGIN_vector.fit(df_5).transform(df_5)

DEST_indexer = StringIndexer(inputCol="DEST", outputCol="DEST_Index")
df_7 = DEST_indexer.fit(df_6).transform(df_6)

onehotencoder_DEST_vector = OneHotEncoder(inputCol="DEST_Index", outputCol="DEST_vec")
df_8 = onehotencoder_DEST_vector.fit(df_7).transform(df_7)
df_8.show(3)

2022-12-16T13:10:05,982 WARN [Thread-4] org.apache.spark.sql.catalyst.util.package - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+-------------------+----------+-----------------+------+----+-------------+---------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------+--------+------+------+----------------+--------------+-----------------------+---------------------+------------+----------------+----------+----------------+
|            FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEPT_TIME|DEPT_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|FL_YEAR|FL_MONTH|FL_DOM|FL_DOW|OP_CARRIER_Index|OP_CARRIER_vec|OP_CARRIER_FL_NUM_Index|OP_CARRIER_FL_NUM_vec|ORIGIN_Ind

In [4]:
vectorAssembler = VectorAssembler(inputCols = ['OP_CARRIER_vec', 'OP_CARRIER_FL_NUM_vec', 'ORIGIN_vec','DEST_vec',
                                               'TAXI_OUT', 'TAXI_IN', 'CANCELLED','DIVERTED','CRS_ELAPSED_TIME',
                                               'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'FL_YEAR', 'FL_MONTH',
                                               'FL_DOM', 'FL_DOW'], outputCol = 'features')
v_df = vectorAssembler.transform(df_8)
v_df.show(3)

+-------------------+----------+-----------------+------+----+-------------+---------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------+--------+------+------+----------------+--------------+-----------------------+---------------------+------------+----------------+----------+----------------+--------------------+
|            FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEPT_TIME|DEPT_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|FL_YEAR|FL_MONTH|FL_DOM|FL_DOW|OP_CARRIER_Index|OP_CARRIER_vec|OP_CARRIER_FL_NUM_Index|OP_CARRIER_FL_NUM_vec|ORIGIN_Index|      ORIGIN_vec|DEST_Index|        DEST_vec|            features|
+-------------------+----------+-----------------+------+----+-------------+---------+---------+--------+----------+---------+-------+--------

In [5]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(labelCol="ARR_DELAY", featuresCol="features")

In [6]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, rf])

In [7]:
from pyspark.ml.tuning import ParamGridBuilder
import numpy as np
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [int(x) for x in np.linspace(start = 10, stop = 50, num = 3)]) \
    .addGrid(rf.maxDepth, [int(x) for x in np.linspace(start = 5, stop = 25, num = 3)]) \
    .build()

In [8]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=RegressionEvaluator(),
                          numFolds=3)

In [9]:
(train_df, test_df) = df_8.randomSplit([0.7, 0.3])

In [ ]:
cvModel = crossval.fit(train_df)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/amcdaid/WORK/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/amcdaid/WORK/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/amcdaid/WORK/spark-3.3.1-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
Traceback (most recent call last):
  File "/home/amcdaid/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code,

In [ ]:
predictions = cvModel.transform(test_df)